# Neural net speech recognition for Videos
Carga de librerías y variables globales

In [2]:
pip install git+https://github.com/openai/whisper.git

The history saving thread hit an unexpected error (OperationalError('database or disk is full')).History will not be written to the database.
Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-cifp08x9
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-cifp08x9
  Resolved https://github.com/openai/whisper.git to commit ba3f3cd54b0e5b8ce1ab3de13e32122d0d5f98ab
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached numba-0.59.0-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (2.7 kB)
  Using cached tiktoken-0.6.0-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.6 kB)
Using cached numba-0.59.0-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (3.7 

In [3]:
import subprocess
import moviepy.editor as mp
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
import whisper
# https://openai.com/blog/whisper/
# pip install git+https://github.com/openai/whisper.git, not whisper.py unrelated package
from pathlib import Path
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
import torch
import re

model_name = 'Falconsai/text_summarization'#'google/pegasus-xsum' #'facebook/bart-large-cnn'
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name).to(torch_device)
# https://arxiv.org/pdf/1912.08777.pdf
# https://medium.com/codex/build-an-automatic-abstractive-text-summarizer-in-ten-minutes-f15f07e54bae

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'T5Tokenizer'. 
The class this function is called from is 'PegasusTokenizer'.
You are using a model of type t5 to instantiate a model of type pegasus. This is not supported for all configurations of models and can yield errors.
Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at Falconsai/text_summarization and are newly initialized: ['decoder.embed_positions.weight', 'decoder.layer_norm.bias', 'decoder.layer_norm.weight', 'decoder.layers.0.encoder_attn.k_proj.bias', 'decoder.layers.0.encoder_attn.k_proj.weight', 'decoder.layers.0.encoder_attn.out_proj.bias', 'decoder.layers.0.encoder_attn.out_proj.weight', 'decoder.layers.0.encoder_attn.q_proj.bias', 'decoder.layers.0.encoder_attn.q_proj.weight', 'decoder.layers.0.encoder_attn.v_proj

Función para determinar la duración total del video

In [3]:
def get_length(filename):
    result = subprocess.run(["ffprobe", "-v", "error", "-show_entries",
                             "format=duration", "-of",
                             "default=noprint_wrappers=1:nokey=1", filename],
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT)
    return float(result.stdout)

Función para generar los subtitulos del video, separando en bloques de chunk_size segundos. La función se puede limitar para que solo genere los subtitulos entre los segundos start y stop, pudiendo usarse como:

-VideoToSRT("video.mp4")

-VideoToSRT("video.mp4", 10)

-VideoToSRT("video.mp4", 10, 52)

-VideoToSRT("video.mp4", 5, 34, 212)

In [16]:
def VideoToSRT(filename, chunk_size=5, start=0, stop=-1):
    num_seconds_video= int(get_length(filename))+1
    print("The video is {} seconds".format(num_seconds_video))
    l=list(range(0,num_seconds_video+1,chunk_size))
    rank=range(len(l)-1)
    if stop>num_seconds_video:
        print("Límite de tiempo superior excede la duración del video, se considerará el final del video")
    if start>0:
        emp=int(start/chunk_size)
    else:
        emp=0
    if stop>-1:
        term=int(stop/chunk_size)+1
    else:
        term=len(l)-1
    rank=range(emp,term)
    diz={}

    for i in rank:
        ffmpeg_extract_subclip(filename, l[i]-2*(l[i]!=0), l[i+1], targetname="chunks/cut{}.mp4".format(i+1))
        clip = mp.VideoFileClip(r"chunks/cut{}.mp4".format(i+1)) 
        clip.audio.write_audiofile(r"converted/converted{}.wav".format(i+1))

        print("Paso 1, i=", i)
        model = whisper.load_model("base")
        result = model.transcribe("converted/converted{}.wav".format(i+1))
        print("Paso 2, i=", i)
        print(round((i-emp)*100/len(rank),2),"%:")
        print(result["text"])
        diz['chunk{}'.format(i+1)]=result["text"]

        l_chunks=[str(i+1)+'\n00:'+"{:02d}".format(int(chunk_size*i/60))+':'+"{:02d}".format(chunk_size*i%60)+',000 --> '+'00:'+"{:02d}".format(int(chunk_size*(i+1)/60))+':'+"{:02d}".format(chunk_size*(i+1)%60-1)+',999\n'+diz['chunk{}'.format(i+1)] for i in range(min(rank),(len(diz)+min(rank)-1))]
        text='\n\n'.join(l_chunks)

        with open(filename[:-3]+'txt',mode ='w') as file:
          file.write("\n")
          file.write(text)
        
        p = Path(filename[:-3]+'txt')
        p.rename(p.with_suffix('.srt'))
    print("100%")
    print("Finally ready!")

## Genera el script del video
Función para obtener el script completo del video.

In [5]:
def VideoScript(filename):
    clip = mp.VideoFileClip(filename)
    clip.audio.write_audiofile(filename[:-3]+".wav")
    model = whisper.load_model("base")
    result = model.transcribe(filename[:-3]+".wav")
    Script=result["text"]
    with open('Script.txt',mode ='w') as file:
          file.write(Script)
    print(Script)
    return(Script)

## Crea un resumen a partir del script
Define las funciones necesarias para resumir el Script en un máximo de max_len oraciones

In [6]:
def AbstractiveSummarization(Script, max_len=5):
    sentences = Script.split(". ")
    textsize=len(sentences)
    paragraph_size=int(textsize/(max_len))
    paragraphs_lims=list(range(0,len(sentences)+1,paragraph_size))
    paragraphs_lims[-1]=textsize
    paragraphs=[]
    main_ideas=[]
    batch = tokenizer.prepare_seq2seq_batch(Script, truncation=True, padding='longest',return_tensors='pt')
    translated = model.generate(**batch)
    tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
    main_ideas.append(tgt_text)
    for i in range(1,len(paragraphs_lims)):
        paragraphs.append('. '.join(sentences[paragraphs_lims[i-1]:paragraphs_lims[i]]))
    for i in range(len(paragraphs)):
        print(min(round((i+1)*100/max_len,2),100),"%")
        batch = tokenizer.prepare_seq2seq_batch(paragraphs[i], truncation=True, padding='longest',return_tensors='pt')
        translated = model.generate(**batch)
        tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
        tgt_text[0] = tgt_text[0].replace('datememe','')
        tgt_text[0] = tgt_text[0].replace('DropCatch','')
        tgt_text[0] = tgt_text[0].replace("Here's the full interview:","")
        if 'n our series of' in tgt_text[0]:
            Names=re.findall(r"[A-Z][a-z]+,?\s+(?:[A-Z][a-z]*\.?\s*)?[A-Z][a-z]+", tgt_text[0])
            if len(Names)>0:
                presence=True
                for j in range(len(Names)):
                    if Names[j] not in paragraphs[i]:
                        presence=False
                if presence==False:
                    print("Resumen incorrecto: ", tgt_text[0])
                    print("Texto original: ", paragraphs[i])
        main_ideas.append(tgt_text)
    return(''.join(idea[0] for idea in main_ideas))

Ejecutando las funciones

La siguiente función unicamente crea los subtitulos del video en un formato SRT para probar la librería Whisper

In [23]:
VideoToSRT("How Clicking a Single Link Can Cost Millions.mp4",5,0,180)

The video is 861 seconds
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
MoviePy - Writing audio in converted/converted1.wav


MoviePy - Done.
Paso 1, i= 0


AttributeError: module 'whisper' has no attribute 'load_model'

La función VideoScript genera e imprime el guión del video completo

In [7]:
Trends=VideoScript("Trends, Opportunities and Challenges.mov")

MoviePy - Writing audio in Trends, Opportunities and Challenges..wav


MoviePy - Done.
 Hello everyone and welcome to another episode of Dojo Live this Tuesday December 6th, 2022. My name is Kim Lantis and it's my pleasure to be co-hosting along with America girl. Hello, it's been one week I think. I know, last week was a bit of a slow week on Dojo Live, it happens. But you know what, we're going to kick it off, we're going to reignite the energy here with SLOV. SLOV, it's so nice to meet you today and thank you for joining us on Dojo Live. Yeah, thanks for heading me. Yes, SLOV, Zulu, Zulu, Zulu Dev. I hope I got that all right. You were the co-host. Yeah, good. Yes. The co-founder and CTO at SOM, so joining us all the way from Germany today, what time is it for you? It's 7 p.m right now. Not too late. Just in time to wrap up your day. Thank you so much for that. We're definitely looking forward to today's show because it's timely. It's all about what's happening in this new year coming up 2023 and something that's pretty important to almost every single

# Resumen

In [41]:
AbstractiveSummarization(Trends, 40)

2.5 %
5.0 %
7.5 %
10.0 %
12.5 %
15.0 %
17.5 %
20.0 %
22.5 %
25.0 %
Resumen incorrecto:  In our series of interviews with the world's leading cryptographers, Koyce's co-founder and chief executive Peter Tonstad reflects on the challenges of building a successful digital currency.
Texto original:  I can talk about this a little bit later about the particular examples. And I mean, the business of Koyce and this online user verification is relatively new. And like technologies change, right? And then also the regulations change, also like crypto emerges. And then suddenly governments realize that they need to control more for obvious reasons, right? So therefore it's really challenging. So I thought like we implement this or that thing and like we'll get eaten. No. It gets more and more complicated
27.5 %
30.0 %
32.5 %
Resumen incorrecto:  In our series of letters from African journalists, film-maker and columnist Ahmedou Ould-Abdallah looks at some of the challenges facing the continent's

Resumen incorrecto:  In our series of letters from Indian journalists, film-maker and columnist Sharmila Tagore reflects on the changing face of journalism:
Texto original:  And the second thing is actually education. Some people are getting more and more used to those verifications. And even like my friends that are even non-technical, they know what KYC is. That was, which we recently surprised. I'm keep hearing that true there, but if not, what it is, right? So, and it will get better and better. So people will ask us, go ask and ask questions. And then, of course, they will trade convenience for, for what's a anonymity
80.0 %
Resumen incorrecto:  In our series of interviews with the world's most powerful technology companies, Facebook's Sheryl Sandberg talks about the biggest challenge facing the social network today:
Texto original:  I mean, it's already started a long time ago, when we will start to use social networks and will be even worse in this respect. But the uploading doc

"In this episode of Dojo Live, Kim talks to SLOV, the co-founder and chief technology officer at verification company SOM.In this week's episode of Dojo Live, Kim and America's own, SLOV, take a look back at some of the best moments from this year's World Taekwondo Championships.You were the co-host.Good morning.In our series of letters from African journalists, film-maker and columnist M'Baye M'Baye reflects on how he came to work for Research Gate, a social network for scientists.In an interview with the BBC's Laura Kuenssberg, Apple's co-founder Tim Cook explains how he came up with the idea for Apple Photos:In our series of interviews with the world's most powerful technology companies, we're going to take a look at how Blockchain is changing the way we do business.How do you protect your users from fraud?I've been trying to set up a bank account in the United States, and I can't.Here's a transcript of the interview:In our series of interviews with the world's leading cryptographer

In [55]:
Surprising=VideoScript("Surprising Things We Have Learned About How Healthcare Works - Joseph Schneier.mp4")
AbstractiveSummarization(Surprising, 40)

MoviePy - Writing audio in Surprising Things We Have Learned About How Healthcare Works - Joseph Schneier..wav


MoviePy - Done.
 Hello everyone and welcome to the final Dojo live show of the week this Thursday November 17th 2022. My name is Ken Lentus and it's my pleasure to be hosting along today with Jorge Hernandez. Jorge, thank you so much for joining me today. Thank you for inviting me to come. It's so much fun to be here. Thank you. It's going to be a fun one and of course the guest of today's show, the most important person joining us today is Joseph Schneier, who is the CEO and co-founder of trusty care. Joseph, thank you so much for spending your time with us today and of course sharing your knowledge with us today. I'm so glad to be here. Yeah, so it's going to be a great conversation. I'm a bit jealous. Joseph is in Las Vegas so he's throughout good food and all that jazz but he's already going to home, I guess. You go to you frequent Vegas too often with work I hear. Well, just enough. All right, so those of you who are wondering what are we going to be talking about today? We're goi

2.5 %
5.0 %
7.5 %
10.0 %
12.5 %
15.0 %
17.5 %
20.0 %
22.5 %
25.0 %
27.5 %
30.0 %
32.5 %
35.0 %
37.5 %
40.0 %
Resumen incorrecto:  In our series of letters from African journalists, film-maker and columnist Ahmed Rashid looks back at some of his most memorable contributions.
Texto original:  And so about, let's see, in 2006 or so, some behavioral research scientists came to me and said, you know, you're doing all this work in education. I wonder if we couldn't apply some of what you're doing to patient behavioral change, which I didn't really have any experience in the healthcare sector at the time
42.5 %
45.0 %
47.5 %
50.0 %
52.5 %
55.0 %
57.5 %
60.0 %
62.5 %
65.0 %
67.5 %
70.0 %
Resumen incorrecto:  In our series of letters from African journalists, filmmaker and columnist Farai Sevenzo looks at some of the top stories of the week.
Texto original:  Wow. Well, thank you so much for sharing that very powerful personal story
72.5 %
75.0 %
77.5 %
80.0 %
Resumen incorrecto:  In our series 

'In this week\'s Dojo, we speak to Joseph Schneier, the CEO and co-founder of trusty care.In this week\'s Dojo, Ken and Jorge discuss the latest news from the world of mixed martial arts.Today, I\'m joined by Jorge Luis Borges, editor-in-chief of La Stampa, the Italian newspaper..On today\'s show we\'re going to be taking a look at some of the top trending topics on social media today.I\'m so happy to be here.All images are copyrighted.What do you think?We\'re going to be talking about how healthcare works.We\'re going to be looking at the big issues of the day.It\'s been a while since I\'ve had a chance to speak to you, but I\'d like to thanks you for taking the time to write to me.I\'ve been running trusty care for four years and I\'ve been running technology companies since 2003I\'ve been a nurse for a long time.How did you get into education?It\'s been a long time since I\'ve written about foreign language teaching.Do you think it\'s possible to learn a foreign language?In our seri

In [56]:
Revolutionizing=VideoScript("Revolutionizing Media Copyright & Ownership for a Prosperous Future on Blockchain - Seagram Pearce.mp4")
AbstractiveSummarization(Revolutionizing, 40)

MoviePy - Writing audio in Revolutionizing Media Copyright & Ownership for a Prosperous Future on Blockchain - Seagram Pearce..wav


MoviePy - Done.
 Hello everyone and welcome to another episode of Dojo Live this Wednesday, November 23rd, 2022. My name is Kim Lantis and it's my pleasure to be hosting along with America Guerrero today. Hello Kim, hello, Sigram. Sigram, right? Yes, Sigram, that's right. Hello, America. Hello, Sigram. We're so happy to have you on the show today. For those of you watching, Sigram is the CEO of Stockably joining us from South Africa. So good evening, Sigram. Yes, good evening. Perfect. You know, so today we're going to be talking about an interesting topic that's, you know, been quite trendy for the last few years and that is blockchain, but specifically how that relates to media copyright. Before we get into the details, however, we would love to get to know you a bit better, Sigram. Your story, your passions, what's led up to Stockably, so please share a bit about yourself with us. Thank you. Thanks, Tim. Well, I've been a creator for probably as long as I can remember. When I left h

2.5 %
5.0 %
7.5 %
10.0 %
12.5 %
15.0 %
Resumen incorrecto:  In our series of letters from African journalists, film-maker and columnist Ahmedou Ould-Abdallah looks at how photography has changed his life.
Texto original:  I just kind of thought I knew what I was doing. So I convinced someone that I knew what I was doing. And in my own, you did that for about two years. And that's when photography sort of started coming in
17.5 %
Resumen incorrecto:  In our series of letters from African journalists, film-maker and columnist Sharmila Tagore reflects on her career as a designer.
Texto original:  And this was something that I, as a kid, I was had dabbled with, but I never sort of realized like this was an actual career option. So that's why I thought design was the creative outlates. Anyway, and so I had a fantastic boss and he gave me an ultimatum and he said, you know, it's a dream you can either quit or I'm going to fire you, but you need to go and do your own thing. And it was kind of

'In this week\'s Dojo Live, we\'re talking about blockchain and how it relates to media copyright.In this week\'s episode of Dojo Live, Kim and America are joined by Sigram for an in-depth discussion of the latest news from the world of mixed martial arts.TRANSCRIPT:Sigram, thank you so much for joining us.Sigram is the co-founder of Stockably, a start-up that aims to make it easier for people to buy and sell stocks online.How long have you been a creator?In our series of letters from African journalists, film-maker and columnist Ahmedou Ould-Abdallah looks at how photography has changed his life.In our series of letters from African journalists, film-maker and columnist Sharmila Tagore reflects on her career as a designer.I\'ve always been passionate about photography.If you want to be a professional photographer, you have to be jack-able.I\'m going to get a dog.In our series of letters from African journalists, film-maker and columnist Ahmed Rashid reflects on his career:In case you 

In [57]:
DataDriven=VideoScript("Data-Driven Approaches to Climate Management - Ryan Meyers.mp4")
AbstractiveSummarization(DataDriven, 40)

MoviePy - Writing audio in Data-Driven Approaches to Climate Management - Ryan Meyers..wav


MoviePy - Done.
 Hello everyone and welcome to another episode of Dojo Live this Thursday's at 10th of January 1st, 2022. It is the final show of the week and the first show of the month. How about that? My name is Kim Lantas and it's my pleasure to be hosting today. Joining me is co-host is Colleen Dibble. Hi Colleen. Hi Colleen. Oh and Matt who surprised us he just popped in. Look at that. What do you call it? Late with style or something like that Matt. I was running back from the A's. Cool. Well thank you for joining us Matt to co-host today and of course the star of today's show is Ryan Myers head of product and data science right at manifest climate. Thank you for joining us Ryan. Yeah my pleasure. Said to be here. Yeah all the way in Toronto. So hey I'm kidding. That is a thing. Yeah. I really enjoyed the moves from Brother Bear they got me gay going. So yes so today we're going to be talking about data driven approaches to climate management. Before we get into that topic howev

2.5 %
5.0 %
7.5 %
10.0 %
12.5 %
15.0 %
Resumen incorrecto:  In our series of letters from African-American journalists, film-maker and columnist John Legend reflects on how he got into the world of software.
Texto original:  And that was kind of like an ignorance bliss moment of my life where I joined to start up with two other guys in the mid 50s who had been consultants for you know 30 years. None of us knew anything about software. And but we knew that we wanted to democratize what we called sustainable return on investment analysis or kind of business case analysis for the built environment so we could basically make the case for greener infrastructure greener buildings. So I was at that company for four years and learned a lot about how to build and how not to build products and software which was pretty cool. And then transitioned into data science actually from there. So with a bio degree in an MBA I think a lot of people actually in product and software take very non-linear pat

Resumen incorrecto:  In our series of interviews with the world's leading experts on climate change, former UN climate chief Christiana Figueres reflects on the shift in the public's attitude towards the issue.
Texto original:  This just wasn't as important. But what we're seeing now, and this was kind of accelerated by COVID as well because people are realizing that these things, these risks are actually not so far off and actually were more interconnected than we maybe gave credit for and like all these different things. And the climate is starting to change like the narrative is changing from this is going to happen to holy crap, this is happening. Right. So with all of that at play at the same time and you're seeing buying patterns shift. There's still it's not as mainstream, I think as it should be, but I can tell you we don't have to educate as much anymore because there are a lot more companies out there where it's not just the sustainability team that's saying we need this. It'

"In today's Dojo Live we're joined by Ryan Myers head of product and data science at manifest climate.In this week's Dojo Live we take a look at the latest news from the world of mixed martial arts.Today we're going to be looking at climate change.Yeah I'm here.On today's show we're looking at climate change and what we can do about it.I was born and raised in the UK.In our series of letters from African-American journalists, film-maker and columnist John Legend reflects on how he got into the world of software.In our series of letters from Canadian business leaders we're looking at some of the stories behind some of the country's most successful companies.In our series of letters from African journalists, filmmaker and columnist Ahmedou Ould-Abdallah looks at the impact of climate change on the lives of ordinary people.In our series of letters from African journalists, filmmaker and columnist Farai Sevenzo looks at how companies can prepare for the risks posed by climate change.Here i

In [58]:
Making=VideoScript("Making Tech Accessible for Small-to-Midsized Restaurants.mp4")
AbstractiveSummarization(Making, 40)

MoviePy - Writing audio in Making Tech Accessible for Small-to-Midsized Restaurants..wav


MoviePy - Done.
 Hello everyone and welcome to another episode of Doja Live, our first episode this week, Tuesday November 15th, 2022. My name is Kim Lentus and it's my pleasure to be hosting along with America Guerrero today. Hello, good morning, good afternoon, good evening for those for the other set of the war. Hello everybody. Exactly. Hello and of course we're here to welcome Sterling Douglas who is the co-founder and CEO of Chowly today, the most important person on today's show. Thank you for joining us Sterling. Thanks for having me Kim. Yeah, it's gonna be a fun conversation all about food. If you haven't picked that up already by the name of the company, Chowly and we're going to be talking about tech and restaurants and practical insights for those of you who want to get started, but before we do, we'd love to get to know you a bit more Sterling. If you could share us, share with us a bit about who you are, your story, your passion and how that's led up to Chowly. Thank you

2.5 %
5.0 %
7.5 %
10.0 %
Resumen incorrecto:  In our series of letters from African-American journalists, novelist and writer Ta-Nehisi Coates reflects on his career:
Texto original:  I was glad that the movie existed, so at least people had an idea of what an actuary might be. But I had a great job. I had really cool clients I worked on. The work was pretty interesting, but there was always this little question that every project we'd work on would start with. It would be kind of what did we do last time? We'd take that and we'd iterate and then we'd kind of deliver that to our clients. And so everything I worked on was kind of a Frankenstein monster. I never got to work on something I really knew
12.5 %
15.0 %
17.5 %
20.0 %
Resumen incorrecto:  In our series of letters from African journalists, film-maker and columnist Ahmed Rashid looks at some of the most striking stories from the continent this year.
Texto original:  So you actually get the right. Wow. So that makes a lot of sense

'Sterling Douglas, the co-founder and CEO of Chowly, the food ordering app, joins Doja Live to talk about how his passion for food led him to start his own company.Kim Lentus: Doja LiveKim: Hello, my name\'s Kim and I\'m the co-founder of Chowly and I\'m here to talk to you about food.How did you get into video games?In our series of letters from African-American journalists, novelist and writer Ta-Nehisi Coates reflects on his career:When I was working at a start-up, we always had a bunch of different problems that we were solving.In this week\'s episode of Pitch Battle, we\'re talking to Charlie Lee, the co-founder of Charlie, a restaurant technology company.Grubhub\'s co-founder, Matt Maloney, explains how the food ordering service is integrating with restaurants\' point of sale systems.In our series of letters from African journalists, film-maker and columnist Ahmed Rashid looks at some of the most striking stories from the continent this year.On today\'s show, we\'re going to be l

In [59]:
Revolutionizing=VideoScript("Revolutionizing Media Copyright & Ownership for a Prosperous Future on Blockchain - Seagram Pearce.mp4")
AbstractiveSummarization(Revolutionizing, 40)

MoviePy - Writing audio in Revolutionizing Media Copyright & Ownership for a Prosperous Future on Blockchain - Seagram Pearce..wav


MoviePy - Done.
 Hello everyone and welcome to another episode of Dojo Live this Wednesday, November 23rd, 2022. My name is Kim Lantis and it's my pleasure to be hosting along with America Guerrero today. Hello Kim, hello, Sigram. Sigram, right? Yes, Sigram, that's right. Hello, America. Hello, Sigram. We're so happy to have you on the show today. For those of you watching, Sigram is the CEO of Stockably joining us from South Africa. So good evening, Sigram. Yes, good evening. Perfect. You know, so today we're going to be talking about an interesting topic that's, you know, been quite trendy for the last few years and that is blockchain, but specifically how that relates to media copyright. Before we get into the details, however, we would love to get to know you a bit better, Sigram. Your story, your passions, what's led up to Stockably, so please share a bit about yourself with us. Thank you. Thanks, Tim. Well, I've been a creator for probably as long as I can remember. When I left h

2.5 %
5.0 %
7.5 %
10.0 %
12.5 %
15.0 %
Resumen incorrecto:  In our series of letters from African journalists, film-maker and columnist Ahmedou Ould-Abdallah looks at how photography has changed his life.
Texto original:  I just kind of thought I knew what I was doing. So I convinced someone that I knew what I was doing. And in my own, you did that for about two years. And that's when photography sort of started coming in
17.5 %
Resumen incorrecto:  In our series of letters from African journalists, film-maker and columnist Sharmila Tagore reflects on her career as a designer.
Texto original:  And this was something that I, as a kid, I was had dabbled with, but I never sort of realized like this was an actual career option. So that's why I thought design was the creative outlates. Anyway, and so I had a fantastic boss and he gave me an ultimatum and he said, you know, it's a dream you can either quit or I'm going to fire you, but you need to go and do your own thing. And it was kind of

'In this week\'s Dojo Live, we\'re talking about blockchain and how it relates to media copyright.In this week\'s episode of Dojo Live, Kim and America are joined by Sigram for an in-depth discussion of the latest news from the world of mixed martial arts.TRANSCRIPT:Sigram, thank you so much for joining us.Sigram is the co-founder of Stockably, a start-up that aims to make it easier for people to buy and sell stocks online.How long have you been a creator?In our series of letters from African journalists, film-maker and columnist Ahmedou Ould-Abdallah looks at how photography has changed his life.In our series of letters from African journalists, film-maker and columnist Sharmila Tagore reflects on her career as a designer.I\'ve always been passionate about photography.If you want to be a professional photographer, you have to be jack-able.I\'m going to get a dog.In our series of letters from African journalists, film-maker and columnist Ahmed Rashid reflects on his career:In case you 

In [60]:
Evolution=VideoScript("The Evolution of Conversational AI - Jaya Kishore.mp4")
AbstractiveSummarization(Evolution, 40)

MoviePy - Writing audio in The Evolution of Conversational AI - Jaya Kishore..wav


MoviePy - Done.
 Hello everyone and welcome to another episode of Dojo Live this Tuesday, November 8th, 2022. My name is Kim Lantis and it's my pleasure to be hosting a long with America Guerrero. Hello everybody, hello Jaya, Jaya, Kishore, ready? Jaya and Kishore, we're going to be calling you Kishore today and for your request. We're so happy to have you on our show, CoFounder and CTO of Yellow AI joining us all the way from Abu Dhabi 10 PM. So Kishore, you are a trooper. Thank you so much. Hey, thank you. Kim, thanks America. Thank you for having me here today. It's going to be a great conversation. We already got to banter a bit pre-show and I know you have quite the story to tell. So before we move on specifically to Yellow AI and more specifically conversational AI, we'd like to get to know you a bit better Kishore. If you could please just share a bit of your story, your passion and sort of what's led up to Yellow. Thank you. Awesome. Yeah, that's definitely a nice story to talk

2.5 %
5.0 %
7.5 %
10.0 %
12.5 %
Resumen incorrecto:  In our series of letters from African journalists, filmmaker and columnist M Ilyasova catches up with the co-founder and chief executive of messaging service WhatsApp, Jan Koum.
Texto original:  So in around 2014, I started one of the co-founder, Ragu and Raksit, the CTO co-founders. So we started in 2015 on solving the very unique problem of the kind of connecting businesses and users over chat or messaging. So in 2015, WhatsApp, especially in emerging markets like India and Southeast Asia and messaging was a huge thing. Everybody was on WhatsApp and Facebook Messenger and stuff like that. So very interesting to know is that even our grandparents used to, the first time they have never exposed to computers and first time they've been exposed to the mobile. So they started, they understand messaging that anything else
15.0 %
17.5 %
20.0 %
Resumen incorrecto:  In our series of interviews with technology start-ups, we speak to the co-f

77.5 %
80.0 %
82.5 %
Resumen incorrecto:  In our series of interviews with the world's leading technology companies, Microsoft's Satya Nadella explains how the company's artificial intelligence platform, Cortana, is being used by businesses around the world.
Texto original:  I need to build a chatbot for those your life. I can just go sign up on the platform and you can actually, sorry, is my connectivity okay? Yeah, you're connectivity is just fine. Okay, so, by the version 3 of the platform, you will be able to create, now you will be able to create use cases, drag and drop, you can build as a flow chat and then deploy it. And now the language understanding has become so mature that it has learned over billions of conversations that have happened on our platform and with real world use cases. So, the language understanding is much more mature and much more global in that way. I have a question
85.0 %
87.5 %
90.0 %
92.5 %
95.0 %
97.5 %
Resumen incorrecto:  In our series of letters fro

"In this week's episode of Dojo Live, co-founder and chief technology officer (CTO) of Yellow AI, Kishore Rao, talks about his company's conversational artificial intelligence platform.Dojo Live with America GuerreroHere's how it went:Hi, I'm Kishore and I'm one of the co-founders of Yellow AI.I've been working in the software industry for the last 10 years.In our series of letters from African journalists, filmmaker and columnist M Ilyasova catches up with the co-founder and chief executive of messaging service WhatsApp, Jan Koum.When we started out, we had a bunch of young people who wanted to start their own business.The number of people using Facebook every day has more than doubled in the past eight years.In our series of interviews with technology start-ups, we speak to the co-founder and chief executive of artificial intelligence (AI) firm DeepMind, Deepa Mehta.In our series of letters from African-American journalists, film-maker and columnist Richard Roeper looks at some of th

In [6]:
Generative=VideoScript("Generative AI for Content Marketing - Sharmin Ali.mp4")
AbstractiveSummarization(Generative, 40)

MoviePy - Writing audio in Generative AI for Content Marketing - Sharmin Ali..wav


MoviePy - Done.
 Hello everyone and happy new year. Welcome to the very first episode of 2023. This January 3rd we're very very happy to be back here live on Dojo Live. My name is Kim Lantis and it's my pleasure to be hosting along with America Guerrero. Hey happy new year everybody. You're happy new year and of course we're kicking the year up with a bang with none other than Charmine Oli who's joining us all the way from India. It's 11.30 p.m. for her y'all so she is a powerhouse. Thank you Charmine for joining us today. Thank you so much for having me here and happy new year to you Kim to America. I love your name. I do everyone else here and who's got to watch the show. Very very happy to talk about generative AI. Thank you very much for being here. I think it's a really fitting conversation for the first show of the year. I think that's a bit of serendipity. We're all about efficiency. We're all about being effective with those new year solutions. I think what you and InstaOriD ar

2.5 %
5.0 %
7.5 %
10.0 %
12.5 %
15.0 %
17.5 %
20.0 %
22.5 %
25.0 %
27.5 %
30.0 %
32.5 %
35.0 %
37.5 %
Resumen incorrecto:  In our series of letters from British journalists, the BBC's political editor Laura Kuenssberg looks at some of the key stories of the year.
Texto original:  I think it's a really fitting conversation for the first show of the year
40.0 %
42.5 %
45.0 %
47.5 %
50.0 %
52.5 %
55.0 %
Resumen incorrecto:  In our series of letters from Indian journalists, film-maker and columnist Sharmila Tagore reflects on her career.
Texto original:  So you know like most Indians I too am an engineer that too into electronics
57.5 %
60.0 %
Resumen incorrecto:  In our series of letters from Indian journalists, film-maker and columnist Sharmila Tagore looks back at some of her most memorable contributions to the country.
Texto original:  So you know from one of the premier institutions in the southern part of India passed out in 2010 oops you know my age now
62.5 %
65.0 %
67.5 %
Resumen 

"In this episode of Dojo Live we're joined by Charmine Oli the founder and CEO of in-storyed, a generative AI company.All images are copyrighted.BBC Sport takes a look back at some of the most memorable moments from the 2023 Rugby World Cup.Dojo Live is back!On this week's Newsround, I'm joined by America Guerrero and Kim Lantis to discuss the latest boxing news from around the world.Hey happy new year everybody.Happy new year!A look back at some of the most memorable moments from this year's Brit Awards.Check out some of the best moments from this year's Grammy Awards, which were held in Los Angeles on Sunday.As part of our series of letters from African journalists, we're looking at some of the most striking stories from the continent this year.Kim Kardashian West has welcomed in the new year with a video message to her fans.All photographs courtesy of AFP, EPA, Getty Images and ReutersBBC Sport takes a look back at some of the most memorable moments from this year's Six Nations..BBC

In [61]:
import wikipedia as wp
query = "KYC"
wp.summary(query, sentences = 1)

'Know Your Customer (KYC) guidelines in financial services require that professionals make an effort to verify the identity, suitability, and risks involved with maintaining a business relationship.'

In [6]:
Extra=VideoScript("AI and Extra-Financials Reporting_ Exploring the Limits - Paul Allard.mp4")
AbstractiveSummarization(Extra, 40)

MoviePy - Writing audio in AI and Extra-Financials Reporting_ Exploring the Limits - Paul Allard..wav


MoviePy - Done.
 Hello everyone and welcome to another episode of Doja Live this Thursday January 12th, 2023. My name is Kim Lantis and it's my pleasure to be hosting today along with America Guerrero. Hello everybody, exciting for being here. Yes, it's always exciting here on Doja Live because we always get to speak to really amazing individuals in the tech industry and today it's none other than Paul Ellerd who is the co-founder and CEO of Impact. Paul, thank you so much for joining us on Doja Live today. It's a real pleasure. Yes, we're laughing at the differences here between what we're experiencing in Mexico and what you're experiencing in Montreal. Yeah, I'm sorry everybody, so it's worth it. I mean the countryside, it's not quite Montreal. So that's no, and in Mexico, I guess you don't want to have as much. No, snow, I would show you the view out my window, but it's just a parking lot. That's great. Yeah, no snow, but I do miss the snow. And I can only imagine how beautiful this

2.5 %
5.0 %
7.5 %
10.0 %
12.5 %
Resumen incorrecto:  In our series of letters from African journalists, film-maker and columnist Ahmed Rashid looks back at some of his most memorable contributions.
Texto original:  And the story is interesting and I did a few going public transaction and one of those was in 2000. It was the first time fresh out of the MBA in Finance. I thought I knew everything about Finance. And I worked with an investment banker, investment bank for a year prior to us going public. My company called Zach Technology in the internet period. And I was astonished in that my first experience when we went public and I lived it for a few years, I was astonished to learn that what I've learned in school had nothing to do with real life
15.0 %
Resumen incorrecto:  In our series of letters from African journalists, former South African finance minister Trevor Manuel reflects on his time in finance and how it changed his life.
Texto original:  I learned some stuff that was happ

52.5 %
Resumen incorrecto:  In our series of letters from African journalists, filmmaker and columnist Ahmedou Ould-Abdallah looks at the state of the continent.
Texto original:  I was saying it's beautiful in a way. I find it to be extremely hopeful. It is all hopeful, but it doesn't move fast enough. I think that there's still a lot of lack of courage within the managers of these big companies, where they're there sitting way. I'm not sure. I may take a risk
55.0 %
Resumen incorrecto:  In our series of interviews with the world's leading technology and business leaders, Microsoft's Satya Nadella talks about the importance of taking risks.
Texto original:  And yes, if you want to move and take a step in the next step to get on the next level, you always take a risk of falling back. And I'm looking at you guys, when you launched Dojo, where was it risk? To innovate, to change, to transform, to live, there's a risk. So that's the problem of a lot of big organization, where the managers 

Resumen incorrecto:  In our series of letters from African journalists, former South African President Nelson Mandela reflects on his life and career:
Texto original:  It's as simple as that. We would have been like a lot of companies, we would have disappeared, and we asked all of our employees at that time, and we were 25, 30, we said, look, we're in a crash crunch. What do we do? We stop here or we take, we collectively transparency. What can be done and what's your leeway, who can, who cannot, and let's get organized together. That's what we've done. It went through, and today we're 135 people
100.0 %
100 %
100 %
100 %
100 %


KeyboardInterrupt: 

In [12]:
Behavioral=VideoScript("Behavioral Targeting for B2B.mp4")
AbstractiveSummarization(Behavioral, 40)

MoviePy - Writing audio in Behavioral Targeting for B2B..wav


MoviePy - Done.
 Hello everyone and welcome to another episode of Doja Live. This Wednesday, January 11th, 2023. My name is Kim Lantis and joining me is co-host today is America Guerrero. Hello everybody. Hey Sam, how are you? Hey, hey America. And of course the guest of honor today, the most important person on today's show is Sam Fono Imoana, who is the founder and CEO of Data Join. Sam, thank you for being here. Hey thanks for having me on the show. This is my first live show so I'm really happy to be with you guys. Yeah, that's, you know, there's fun in the live show. I think it's just the unexpected can happen, right? So that's, that's perfect. And it's fun. We like to keep it conversational. We like to keep it real. And I think that's something that doesn't always happen in its, you know, produced. So it's part of our spirit here, but perfect. So today we're talking all about that behavior and that spirit with behavioral targeting, particularly for B2B business to business. But b

2.5 %
5.0 %
7.5 %
10.0 %
12.5 %
Resumen incorrecto:  In our series of letters from African-American journalists, filmmaker and columnist Steve Inskeep looks back at some of his earliest memories.
Texto original:  I'm just here in Utah. This is, which is where I live now. And I was, I was studying business over there. But in the midst of learning about, you know, EBITDA and, and leadership classes for business. You know, I took a couple of information systems classes. And, you know, one of them was on Microsoft Excel. And the other one was, yeah, I mean, I didn't even know what a, what a spreadsheet was. I just remember taking that class
15.0 %
17.5 %
20.0 %
Resumen incorrecto:  In our series of letters from African-American journalists, filmmaker and columnist Salil Shetty looks at what it takes to be a successful journalist.
Texto original:  And the reason why I say that is because, you know, all throughout my career, that's why I've helped. I've come in with these with these analysis

90.0 %
92.5 %
95.0 %
Resumen incorrecto:  In our series of letters from African-American journalists, filmmaker and columnist Garry Trudeau reflects on the importance of core values in the workplace.
Texto original:  I mean, we're from the islands and it's this idea that we back each other up. We show a lot of care for for one another and respect. And that's just important like culturally, you know, that's that's our personal backgrounds, right. A lot of us at my company are from are from the islands and so. But the second core value is is just as important, you know, possibly even more important and that's find a way right we always find a way. You know, grit, that's probably a good word to describe it, but you know in these startup atmospheres like that's we also need that just as much. I mean this morning on our company meeting that was that was what we shared this morning on our company meeting is that grit is. Grit is a great prevails whenever you get knocked down and you stand ba

'Sam Fono Imoana, the founder and CEO of Data Join, is the guest of honor on the latest episode of Doja Live.On this week\'s episode of Doja Live, we are joined by Sam Fono Imoana, the founder and CEO of Data Join.Here is the full interview:We\'ve been talking a lot lately about the fact that we\'re living in a world where we\'re more connected than ever before.I\'m a software engineer.In our series of letters from African-American journalists, filmmaker and columnist Steve Inskeep looks back at some of his earliest memories.When I was in high school, I took a class on how to use Excel.When I look back on my career, I\'m very proud of the fact that I\'ve been able to learn so much along the way.In our series of letters from African-American journalists, filmmaker and columnist Salil Shetty looks at what it takes to be a successful journalist.Here\'s what Facebook\'s Sheryl Sandberg had to say about the company\'s new data join feature:On today\'s show, we\'re going to talk a little bit

In [13]:
Finances=VideoScript("Finances Inside and Outside the Metaverse (1).mp4")
AbstractiveSummarization(Finances, 40)

MoviePy - Writing audio in Finances Inside and Outside the Metaverse (1)..wav


MoviePy - Done.
 Good morning everyone and welcome to Dojo Live. My name is Kim Lantis and it's my pleasure to be here moderating what is the part 2 or 4 part series all about the metaverse every Friday for the month of January. Joining me today are the same two of the same individuals that we had last week. We've got D&L Agamboa who is a UX designer as well as for Mean Chavez and experienced researcher both with Encore and today's special guest is Nippur Avantika, VP of Digital Learning at Encore India. Hello everyone. Hi, good morning. Good morning. And thank you to your audience right for joining us a bit earlier today than usual. We were able to accommodate Nippur who's just wrapping up her day as we're getting ours started. Now Nippur before we get started into today's topic about the metaverse all about finances inside and outside. Let's get to know you just a bit better. Please, what's your story, your passion and what's led up to your time focused on digital learning? Sure, Kim

2.5 %
5.0 %
7.5 %
10.0 %
12.5 %
Resumen incorrecto:  In our series of letters from African-American journalists, filmmaker and columnist Steve Inskeep looks back at some of his earliest memories.
Texto original:  I'm just here in Utah. This is, which is where I live now. And I was, I was studying business over there. But in the midst of learning about, you know, EBITDA and, and leadership classes for business. You know, I took a couple of information systems classes. And, you know, one of them was on Microsoft Excel. And the other one was, yeah, I mean, I didn't even know what a, what a spreadsheet was. I just remember taking that class
15.0 %
17.5 %
20.0 %
Resumen incorrecto:  In our series of letters from African-American journalists, filmmaker and columnist Salil Shetty looks at what it takes to be a successful journalist.
Texto original:  And the reason why I say that is because, you know, all throughout my career, that's why I've helped. I've come in with these with these analysis

90.0 %
92.5 %
95.0 %
Resumen incorrecto:  In our series of letters from African-American journalists, filmmaker and columnist Garry Trudeau reflects on the importance of core values in the workplace.
Texto original:  I mean, we're from the islands and it's this idea that we back each other up. We show a lot of care for for one another and respect. And that's just important like culturally, you know, that's that's our personal backgrounds, right. A lot of us at my company are from are from the islands and so. But the second core value is is just as important, you know, possibly even more important and that's find a way right we always find a way. You know, grit, that's probably a good word to describe it, but you know in these startup atmospheres like that's we also need that just as much. I mean this morning on our company meeting that was that was what we shared this morning on our company meeting is that grit is. Grit is a great prevails whenever you get knocked down and you stand ba

'Sam Fono Imoana, the founder and CEO of Data Join, is the guest of honor on the latest episode of Doja Live.On this week\'s episode of Doja Live, we are joined by Sam Fono Imoana, the founder and CEO of Data Join.Here is the full interview:We\'ve been talking a lot lately about the fact that we\'re living in a world where we\'re more connected than ever before.I\'m a software engineer.In our series of letters from African-American journalists, filmmaker and columnist Steve Inskeep looks back at some of his earliest memories.When I was in high school, I took a class on how to use Excel.When I look back on my career, I\'m very proud of the fact that I\'ve been able to learn so much along the way.In our series of letters from African-American journalists, filmmaker and columnist Salil Shetty looks at what it takes to be a successful journalist.Here\'s what Facebook\'s Sheryl Sandberg had to say about the company\'s new data join feature:On today\'s show, we\'re going to talk a little bit